# by *convention* documents should use real data.

__pidgin.template__ supports jinja2 syntax to data in code cells before they execute.

# Templating input cells with [`jinja2`](http://jinja.pocoo.org/docs/2.10/)

This notebook demonstrates and exports the ability to place real into code cells before executing code. This idea can create parameterized reports.

    %reload_ext deathbeds.__Jinja2__

In [21]:
    from .markdown import CallableTransformer, render
    from jinja2 import Template
    from importnb import Interactive

In [22]:
    class TemplateTransformer(CallableTransformer):
        def __call__(self, str):
            from IPython import get_ipython
            return Template(str).render(**get_ipython().user_ns)

## Importer

In [23]:
    class Jinja2Importer(Interactive):
        extensions = '.tpl.ipynb',
        def parse_code_cell(self, cell, module=None): 
            cell['source'] = Template(''.join(cell['source'])).render(**vars(module))
            return super().parse_code_cell(cell)

In [30]:
    class Jinja2MarkdownImporter(Interactive):
        extensions = '.tpl.md.ipynb',
        def parse_code_cell(self, cell, module=None): 
            cell['source'] = render(Template(''.join(cell['source'])).render(**vars(module)))
            return super().parse_code_cell(cell)

In [35]:
    def load_ipython_extension(ip=None):
        ip = get_ipython()
        if ip:
            ip.input_transformer_manager.physical_line_transforms.insert(0, TemplateTransformer())

In [36]:
    def unload_ipython_extension(ip=None):
        ip = ip or get_ipython()
        if ip:
            ip.input_transformer_manager.python_line_transforms = [
                object for object in ip.input_transformer_manager.python_line_transforms
                if not isinstance(object, TemplateTransformer)
            ]

## Testing

In [37]:
    class Test(__import__('unittest').TestCase): 
        def setUp(Test):
            load_ipython_extension()
            from nbformat import write, v4
            with open('test_template.tpl.ipynb', 'w') as file:
                write(v4.new_notebook(cells=[
                    v4.new_code_cell("""if __name__ == '__main__':\t\n%reload_ext template"""),
                    v4.new_code_cell("""a=42"""),
                    v4.new_code_cell("""b={{a}}*10""")
                ]), file)
                
        def runTest(Test):
            global test_template
            import test_template
            assert test_template.__file__.endswith('.ipynb')
            assert test_template.a is 42
            assert test_template.b == 420
            
        def tearDown(Test):
            __import__('os').remove('test_template.tpl.ipynb')
            unload_ipython_extension()

## Developer

In [38]:
    if __name__ == '__main__': 
        reporter = __import__('unittest').TextTestRunner()
        reporter.run(Test())
        __import__('doctest').testmod(verbose=2, report=reporter)

.
----------------------------------------------------------------------
Ran 1 test in 0.008s

OK


14 items had no tests:
    __main__
    __main__.Jinja2Importer
    __main__.Jinja2Importer.parse_code_cell
    __main__.Jinja2MarkdownImporter
    __main__.Jinja2MarkdownImporter.parse_code_cell
    __main__.TemplateTransformer
    __main__.TemplateTransformer.__call__
    __main__.Test
    __main__.Test.runTest
    __main__.Test.setUp
    __main__.Test.tearDown
    __main__.load_ipython_extension
    __main__.parse_code_cell
    __main__.unload_ipython_extension
0 tests in 14 items.
0 passed and 0 failed.
Test passed.
